```bash
conda create --name ani python=3.8
conda install -c conda-forge jupyterlab
pip install open_clip_torch
pip install stable-baselines3
pip install gym[all]
pip install pyglet==1.5.27
```

In [1]:
import gym

from stable_baselines3 import DQN
import open_clip
from PIL import Image
import torch
import numpy as np

/home/nikitafordui/miniconda3/envs/ani/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model, _, preprocess = open_clip.create_model_and_transforms('ViT-B-32-quickgelu', pretrained='laion400m_e32')
tokenizer = open_clip.get_tokenizer('ViT-B-32-quickgelu')

In [3]:
class CLIPEnv():
    def __init__(self, env, clip_model, tokenizer, prompt):
        self.env = env
        self.model = clip_model
        self.tokenizer = tokenizer
        self.text_features = self.model.encode_text(self.tokenizer([prompt]))
        self.text_features /= self.text_features.norm(dim=-1, keepdim=True)
        
        self.action_space = self.env.action_space
        self.observation_space = self.env.observation_space
        self.metadata = self.env.metadata 

    
    def reset(self):
        return self.env.reset()
    
    def step(self, action):
        next_st, rwd, done, info = self.env.step(action)
        img = self.env.render(mode="rgb_array")
        clip_rwd = self.get_clip_reward(img)
        print(clip_rwd)
        return next_st, clip_rwd, done, info
    
    def get_clip_reward(self, state):
        image = preprocess(Image.fromarray(np.uint8(state))).unsqueeze(0)
        with torch.no_grad(), torch.cuda.amp.autocast():
            image_features = self.model.encode_image(image)
            image_features /= image_features.norm(dim=-1, keepdim=True)
            # text_probs = (image_features @ self.text_features.T).softmax(dim=-1)
            sim = (image_features @ self.text_features.T)
        return sim.cpu().detach().numpy()[0][0]

In [4]:
env = gym.make('LunarLander-v2')
cl_env = CLIPEnv(env, model, tokenizer, 'Spaceship is on the landing pad')

agent = DQN('MlpPolicy', cl_env, verbose=1)
agent.learn(total_timesteps=100)

obs = env.reset()
for i in range(100):
    action, _states = agent.predict(obs)
    obs, rewards, dones, info = cl_env.step(action)

env.close()

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
0.2607556
0.24848498
0.25717267
0.25884557
0.24699256
0.25616392
0.26295635
0.2424294
0.25081986
0.2541457
0.26320076
0.25535217
0.2583518
0.26485813
0.26377368
0.26387528
0.26598412
0.26422632
0.26454136
0.2644203
0.2606246
0.2539432
0.25520074
0.2620947
0.2401269
0.25481346
0.25683063
0.25518692
0.2556458
0.25213027
0.25790328
0.24539968
0.23957562
0.24748224
0.2548768
0.2585212
0.2535781
0.26737678
0.2721603
0.26419404
0.25993845
0.25885358
0.26686496
0.2644793
0.25073975
0.26509276
0.2582788
0.26591945
0.26687545
0.26256698
0.26539052
0.2620371
0.26111895
0.26024473
0.25532293
0.26304
0.26436728
0.2596693
0.24875307
0.25965124
0.26203704
0.256956
0.2732224
0.2650956
0.2594818
0.26677105
0.27447546
0.2662105
0.26520398
0.2748747
0.26214993
0.26407474
0.2592531
0.25796095
0.26457906
0.27143335
0.26811546
0.26565775
0.26318052
0.26229137
0.2593961
0.26020038
0.26749957
0.2612287
0.26003146
0